# Rozwiązania do zadań dla bloku Python Średniozaawansowany

## Wyjątki

#### Przechwyć wynik wyjątku wyrzucanego przez funkcję `raiseErr` i wypisz go z pomocą operacji `print`.

##### Rozwiązanie:

 * wykorzystaj klauzulę `try-except`.

 * przechwyć wyjątek za pomocą `except as`.

In [1]:
def raiseErr():
    raise Exception("Hah! You got me!")

try:
    raiseErr()
except Exception as e:
    print(e)

Hah! You got me!


#### Stwórz klasę wyjątku `TimedException`, która, wraz z wiadomością, będzie przechowywała dane o czasie wystąpienia błędu. Skorzystaj z modułu [datetime](https://docs.python.org/3/library/datetime.html).

##### Rozwiązanie:

 * stwórz klasę dziedziczącą po `Exception`.

 * w konstruktorze ustaw pole `time` na aktualny czas pobrany z `datetime.datetime.now()`.

 * przechwyć wyjątek i wypisz jego czas i wiadomość.

In [9]:
from datetime import datetime

class TimedException(Exception):
    def __init__(self, *args: object) -> None:
        super().__init__(*args)
        self.time = datetime.now()

try:
    raise TimedException("Hello")
except TimedException as e:
    print(f"{e.time}: {e}")

2022-12-03 18:37:36.621307: Hello


#### Napisz klasę `AutodestructingFile` która stworzy plik o lokalizacji zadanej w konstruktorze i pozwoli na zapisywanie i odczytywanie z owego pliku. Obiekt tej klasy, gdy użyty w ramach klauzuli `with`, ma usunąć automatycznie plik z dysku w momencie zakończenia działania, niezależnie od tego, czy klauzula zakończyła się poprawnie czy błędem. Należy zastąpić wszystkie wystąpienia `pass` odpowiednim kodem.

##### Rozwiązanie

 * zapisz zmienną `address` w konstruktorze jako pole obiektu. Następnie stwórz pola `reader` i `writer` i przypisz do nich odpowiednio `open(address, "w")` i `open(address, "r")`.

 * w operacji `write` zapisz tekst do pliku odwołując się do pola `self.writer`. Analogicznie zczytaj treść pliku z pomocą `self.reader` w operacji `write`.

 * w operacji `__exit__` zamknij `self.writer` i `self.reader` oraz usuń plik z pomocą operacji `os.path.exists` i `os.remove`.

Wywołanie powinno zakończyć się tekstem *"This code should fail here and dummy.file should not exist"*.

In [11]:
class AutodestructingFile:

    def __init__(self, address) -> None:
        self.address = address
        self.writer = open(address, "w")
        self.reader = open(address, "r")

    def write(self, text):
        self.writer.write(text)

    def read(self):
        return self.reader.read()

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.reader.close()
        self.writer.close()
        if os.path.exists(self.address):
            os.remove(self.address)


dummy_file_name = "dummy.file"
some_dummy_message = "somedummymessage"

assert not os.path.exists(dummy_file_name)
exception = Exception(f"This code should fail here and {dummy_file_name} should not exist")

try:
    with AutodestructingFile(dummy_file_name) as file:
        assert os.path.exists(dummy_file_name)
        file.write(some_dummy_message)
        print(file.read())

        raise exception
        print("This message should not be displayed")
except Exception as e:
    assert e == exception
    print(e)
assert not os.path.exists(dummy_file_name)


This code should fail here and dummy.file should not exist


## Programowanie obiektowe

#### Wczytaj plik za pomocą `reader`, przetwórz treść za pomocą `converter` i zapisz do pliku za pomocą `writer`. Stwórz klasę `LUFileConverter`, która będzie konwertowała zadany plik zmieniając małe litery w wielkie i vice versa. Stwórz klasę `DoubleFileConverter`, która wykona operacje `converter` dwukrotnie.

`convertFile` w `FileConverter` ma być metodą abstrakcyjną.
`reader` ma posiadać operację `read()`, która zczyta obiekt typu `str` z pewnego pliku.
`writer` ma posiadać operację `write()`, która zapisze do pewnego pliku zadany obiekt typu `str`.
`converter` ma przyjąć obiekt typu `str` i zwrócić obiekt typu `str`.

Skorzystaj z modułu [abc](https://docs.python.org/3/library/abc.html).

##### Rozwiązanie

 - Stwórz plik (`dummy.txt` lub jakikolwiek inny) z dowolną treścią (`Ala ma kota` lub cokolwiek innego).
 - Wczytaj treść tego pliku za pomocą operacji `open` i `read`.
 - Zapisz treść tego pliku za pomocą `open` i `write` do innego pliku (`temp.file` lub jakikolwiek inny).
 - Stwórz funkcję `toUpper`, która przyjmie obiekt typu `str` i zwróci ów tekst z wszystkimi małymi literami zamienionymi na wielkie.
 - Stwórz obiekt typu `FileConverter` i zapisz jako jego pola `reader`, `writer` oraz `toUpper`.
 - Zmień pole `convertFile` w `FileConverter` na metodę abstrakcyjną.

In [2]:
from abc import ABC, abstractmethod

def swapCase(text: str) -> str:
    return text.swapcase()

class FileConverter(ABC):
    def __init__(self, reader, writer, converter) -> None:
        self.reader = reader
        self.writer = writer
        self.converter = converter

    @abstractmethod
    def convertFile(self):
        pass

class LUFileConverter(FileConverter):
    def __init__(self, reader, writer) -> None:
        super().__init__(reader, writer, swapCase)

    def convertFile(self):
        content = self.reader.read()
        content = self.converter(content)
        self.writer.write(content)

class DoubleFileConverter(FileConverter):
    def convertFile(self):
        content = self.reader.read()
        content = self.converter(content)
        content = self.converter(content)
        self.writer.write(content)

In [3]:
def doubleText(text):
    return text + text

in_file = "dummy.txt"
out_file = "temp.file"

with open(in_file, "w") as writer:
    writer.write("Ala ma kota\nA sierotka ma rysia\n\n")

with open(in_file, "r") as reader:
    with open(out_file, "w") as writer:
        fc = DoubleFileConverter(reader, writer, doubleText)
        fc.convertFile()

if os.path.exists(in_file):
    os.remove(in_file)
if os.path.exists(out_file):
    os.remove(out_file)

In [4]:
def doubleText(text):
    return text + text

in_file = "dummy.txt"
out_file = "temp.file"

with open(in_file, "w") as writer:
    writer.write("Ala ma kota\nA sierotka ma rysia\n\n")

with open(in_file, "r") as reader:
    with open(out_file, "w") as writer:
        fc = LUFileConverter(reader, writer)
        fc.convertFile()

if os.path.exists(in_file):
    os.remove(in_file)
if os.path.exists(out_file):
    os.remove(out_file)

#### Napisz strukturę o trzech niemutowalnych polach, którą można posortować oraz przechowywać jako element zbioru.

In [5]:
from dataclasses import dataclass

@dataclass(frozen=True, order=True, unsafe_hash=True)
class Triple:
    a: float
    b: float
    c: float

print(sorted([Triple(1, 2, 3), Triple(2, 1, 19), Triple(1, 2, 1), Triple(1, 3, 2)]))
print({Triple(1, 2, 3), Triple(2, 1, 19), Triple(1, 2, 1), Triple(1, 3, 2)})

[Triple(a=1, b=2, c=1), Triple(a=1, b=2, c=3), Triple(a=1, b=3, c=2), Triple(a=2, b=1, c=19)]
{Triple(a=1, b=2, c=1), Triple(a=1, b=3, c=2), Triple(a=1, b=2, c=3), Triple(a=2, b=1, c=19)}


## Serializacja

## Dekoratory

#### Napisz dekorator `doubleResult`, który sprawi, że funkcja dekorowana zwróci wynik przemnożony przez $2$. Udekoruj nim funkcję `returnNumber`.

Rozwiązanie:
 * dekorator powinien zwracać funkcję, która zastąpi funkcję dekorowaną. Napisz definicję funkcji `wrapper` wewnątrz `doubleResult`.
 * funkcja ta nie powinna przyjmować jakichkolwiek argumentów i powinna zwracać wynik funkcji będącej argumentem dekoratora.
 * wartość zwracaną przez `wrapper` przemnóż przez $2$.
 * dekorator powinien na koniec zwrócić `wrapper`.

In [6]:
from random import random

def doubleResult(func):
    def wrapper():
        return 2 * func()
    return wrapper

number = random()

@doubleResult
def returnNumber():
    return number

assert returnNumber() == number * 2

#### Napisz dekorator `multiplyResultBy` będący modyfikacją dekoratora z zadania powyżej. Niech `multiplyResultBy` przyjmuje jeden argument, którym będzie liczba, przez którą zostanie przemnożony wynik funkcji dekorowanej. Udekoruj nim funkcję `returnNumber`.

Rozwiązanie <font size="1" color="gray">(złożone w opisie, bo sam problem niestety jest złożony)</font>:

 * dekorator wywołamy jako `@multiplyResultBy(n)`, czyli sam `multiplyResultBy(n)` jest funkcją, która zwróci dekorator. Stąd dopiero ta funkcja powinna zwrócić dekorator analogiczny do zadania powyżej.

 * w poprzednim zadaniu `doubleResult` zwracał funkcję zastępującą funkcję dekorowaną. Tym razem musimy na podstawie parametru zwrócić dekorator, który przyjmie funkcję dekorowaną i zwróci jej zastępstwo (uff, złożone, co?).

 * <font size="1" color="red">dla matematycznych formalistów</font> dekorator jest postaci $\R\to(\emptyset\to\R)\to\emptyset\to\R$, to jest, pobiera liczbę, przez którą przemnożymy wynik naszej funkcji ($\R\to$) i zwraca funkcję, która bierze funkcję bezparametrową do udekorowania ($(\emptyset\to\R)\to$), ta funkcja zaś zwraca funkcję o tej samej sygnaturze co funkcja dekorowana ($\emptyset\to\R$).

In [7]:
from random import random

def multiplyResultBy(multiplier):
    def dec(func):
        def wrapper():
            return multiplier * func()
        return wrapper
    return dec

number = random()
n = random()

@multiplyResultBy(multiplier=n)
def returnNumber():
    return number

assert returnNumber() == number * n

#### Ponownie napisz dekorator `multiplyResultBy` o działaniu jak w zadaniu powyżej. Tym razem jednak niech dekorator ten będzie w stanie dekorować funkcje o dowolnej ilości parametrów. Udekoruj nim funkcję `multAddPrint`.

Rozwiązanie:
 * `wrapper` powinien przechwycić wszelkie argumenty za pomocą słów-kluczy `*args` i `**kwargs`.

In [8]:
from random import random

def multiplyResultBy(multiplier):
    def dec(func):
        def wrapper(*args, **kwargs):
            return multiplier * func(*args, **kwargs)
        return wrapper
    return dec

number1, number2, number3 = random(), random(), random()
text = "I am working as intended"
n = random()

@multiplyResultBy(multiplier=n)
def returnNumbers(number1, number2, number3, text):
    print(text)
    return number1 * number2 + number3

assert returnNumbers(number1, number2, number3, text) == (number1 * number2 + number3) * n

I am working as intended


#### Napisz dekorator, który przechwyci wyjątek wyrzucany przez funkcję i wypisze jego treść i ślad na wyjściu konsoli (domyślnie wyjątki wypisują się na `stderr`, standardowe wyjście konsoli to `stdout`).

Do zadania wykorzystaj pakiet [traceback](https://docs.python.org/3/library/traceback.html).

##### Rozwiązanie

 * otocz wywołanie funkcji dekorowanej klauzulą `try-except`.
 * w klauzuli `except` wyświetl stos wywołania za pomocą [print_exc](https://docs.python.org/3/library/traceback.html#traceback.print_exc) z pakietu `traceback`.

In [9]:
import traceback

def captureError(func):
    def wrapper():
        try:
            return func()
        except:
            traceback.print_exc()
    return wrapper


@captureError
def function1():
    def function2():
        raise AssertionError("Why index error? Just so")
    print("First print")
    function2()
    print("Second print, should not be printed")

function1()
print("This should be printed")

First print
Traceback (most recent call last):
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_20900/3269307817.py", line 6, in wrapper
    func()
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_20900/3269307817.py", line 17, in function1
    function2()
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_20900/3269307817.py", line 15, in function2
    raise AssertionError("Why index error? Just so")
AssertionError: Why index error? Just so


#### Napisz dekorator `encrypted`, który przyjmuje jako parametry adres pliku oraz funkcję szyfrującą a następnie zapisuje do tego pliku zaszyfrowany wynik funkcji.

In [10]:
from random import shuffle
import string

basis = list(string.printable)
permutation = list(basis)
shuffle(permutation)

dictionary = {}
for i in range(len(basis)):
    dictionary[basis[i]] = permutation[i]

def encryptor(text):
    return ''.join([dictionary[c] for c in text])

In [11]:
dummy_file_name = "dummy.file"

def encrypted(encryptor, address):
    def dec(func):
        def wrapper():
            with open(address, "w") as writer:
                writer.write(encryptor(func()))
        return wrapper
    return dec

@encrypted(encryptor, dummy_file_name)
def strfunc():
    return "Ala ma kota"

strfunc()

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

#### Napisz powyższe zadanie, tym razem jednak dla funkcji, która przyjmuje argumenty

In [12]:
from random import shuffle
import string

basis = list(string.printable)
permutation = list(basis)
shuffle(permutation)

dictionary = {}
for i in range(len(basis)):
    dictionary[basis[i]] = permutation[i]

def encryptor(text):
    return ''.join([dictionary[c] for c in text])

In [13]:
dummy_file_name = "dummy.file"

def encrypted(encryptor, address):
    def dec(func):
        def wrapper(*args, **kwargs):
            with open(address, "w") as writer:
                writer.write(encryptor(func(*args, **kwargs)))
        return wrapper
    return dec

@encrypted(encryptor, dummy_file_name)
def strfunc(text, n):
    return f"My encrypted text is '{text}' and it should occure {n} times after this line\n" + "\n".join([text] * n)

strfunc("Ala ma kota", 5)

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

## Wyrażenia lambda

#### Stwórz $\lambda$-wyrażenie, które, dla parametru `x`, wyznaczy $x^{17}\mod 37$. Stwórz tablicę takich wyników dla kolejnych $10000$ liczb naturalnych.

In [1]:
f = lambda x: x ** 17 % 37
list(map(f, range(10000)))

[0,
 1,
 18,
 25,
 28,
 22,
 6,
 16,
 23,
 33,
 26,
 27,
 34,
 17,
 29,
 32,
 7,
 13,
 2,
 35,
 24,
 30,
 5,
 8,
 20,
 3,
 10,
 11,
 4,
 14,
 21,
 31,
 15,
 9,
 12,
 19,
 36,
 0,
 1,
 18,
 25,
 28,
 22,
 6,
 16,
 23,
 33,
 26,
 27,
 34,
 17,
 29,
 32,
 7,
 13,
 2,
 35,
 24,
 30,
 5,
 8,
 20,
 3,
 10,
 11,
 4,
 14,
 21,
 31,
 15,
 9,
 12,
 19,
 36,
 0,
 1,
 18,
 25,
 28,
 22,
 6,
 16,
 23,
 33,
 26,
 27,
 34,
 17,
 29,
 32,
 7,
 13,
 2,
 35,
 24,
 30,
 5,
 8,
 20,
 3,
 10,
 11,
 4,
 14,
 21,
 31,
 15,
 9,
 12,
 19,
 36,
 0,
 1,
 18,
 25,
 28,
 22,
 6,
 16,
 23,
 33,
 26,
 27,
 34,
 17,
 29,
 32,
 7,
 13,
 2,
 35,
 24,
 30,
 5,
 8,
 20,
 3,
 10,
 11,
 4,
 14,
 21,
 31,
 15,
 9,
 12,
 19,
 36,
 0,
 1,
 18,
 25,
 28,
 22,
 6,
 16,
 23,
 33,
 26,
 27,
 34,
 17,
 29,
 32,
 7,
 13,
 2,
 35,
 24,
 30,
 5,
 8,
 20,
 3,
 10,
 11,
 4,
 14,
 21,
 31,
 15,
 9,
 12,
 19,
 36,
 0,
 1,
 18,
 25,
 28,
 22,
 6,
 16,
 23,
 33,
 26,
 27,
 34,
 17,
 29,
 32,
 7,
 13,
 2,
 35,
 24,
 30,
 5,
 8,
 20,
 3,
 10

## Generatory i iteratory

## Wyrażenia regularne

## Wielowątkowość

## Profilowanie kodu